In [5]:
import re

file = open('input1.txt', 'r')
regex = '(1|2|3|4|5|6|7|8|9|0|one|two|three|four|five|six|seven|eight|nine)'
translation = {'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9}

sum_digits = 0
sum_digits_and_text = 0

for line in file.readlines():
    matches = []
    for i in range(len(line)):
        match = re.search(regex, line[i:])
        if match is not None:
            matches.append(match.group(0))

    first = -1
    first_digit = -1

    last = -1
    last_digit = -1

    for match in matches:
        digit = 0
        try:
            digit = int(match)
            if first_digit == -1:
                first_digit = digit
            last_digit = digit
        except ValueError:
            digit = translation[match]

        if first == -1:
            first = digit
        last = digit

    if first == -1 or last == -1:
        print(matches)
        print(first)
        print(last)
        
    sum_digits += 10*first_digit + last_digit
    sum_digits_and_text += 10*first + last

print(f'The sum of the first and last digit on each line is {sum_digits}')
print(f'The sum of the first and last digit on each line (including text) is {sum_digits_and_text}')
        

The sum of the first and last digit on each line is 53386
The sum of the first and last digit on each line (including text) is 53312


In [6]:
import re
file = open('input2.txt')

max_red = 12
max_green = 13
max_blue = 14

impossible = []
possible = []
powers = []

for line in file.readlines():
    game_match = re.match('Game ([0-9]+): (.+)$', line)
    id = int(game_match.group(1))
    draws = game_match.group(2).split(';')

    min_red = 0
    min_green = 0
    min_blue = 0
    
    for draw in draws:
        red_match = re.search('([0-9]+) red', draw)
        green_match = re.search('([0-9]+) green', draw)
        blue_match = re.search('([0-9]+) blue', draw)

        red = 0 if red_match is None else int(red_match.group(1))
        green = 0 if green_match is None else int(green_match.group(1))
        blue = 0 if blue_match is None else int(blue_match.group(1))

        min_red = red if red > min_red else min_red
        min_green = green if green > min_green else min_green
        min_blue = blue if blue > min_blue else min_blue

        if id not in impossible and (red > max_red or green > max_green or blue > max_blue):
            impossible.append(id)

    if id not in impossible:
        possible.append(id)

    powers.append(min_red*min_green*min_blue)

print(f'The sum of the IDs of the games that were possible is {sum(possible)}')
print(f'The sum of the powers of the minimum sets of cubes is {sum(powers)}')
        

The sum of the IDs of the games that were possible is 2406
The sum of the powers of the minimum sets of cubes is 78375


In [7]:
import re
import numpy

file = open('input3.txt')
schematic = []
part_numbers = []

gears = {}

for line in file.readlines():
    schematic.append(line.strip())

running_number = ''
adjacent_gears = set()
adjacent = False

for i in range(len(schematic)):
    for j in range(len(schematic[i])):
        c = schematic[i][j]
        if c.isdigit():
            running_number += c

            # Check nearby cells for symbols
            for k in range(-1, 2):
                for l in range(-1, 2):
                    try:
                        other_c = schematic[i+k][j+l]
                        if re.search('[^\.0-9]', other_c) is not None:
                            adjacent = True
                        if other_c == '*':
                            adjacent_gears.add((i+k, j+l))
                    except IndexError:
                        pass

        else:
            if adjacent:
                part_numbers.append(int(running_number))
                for gear in adjacent_gears:
                    if gear not in gears:
                        gears[gear] = []
                    gears[gear].append(int(running_number))

            running_number = ''
            adjacent_gears = set()
            adjacent = False

print(f'The sum of all the part numbers is {sum(part_numbers)}')
print(f'The sum of all gear ratios is {sum(numpy.prod(gears[gear]) for gear in gears if len(gears[gear]) == 2)}')

The sum of all the part numbers is 539637
The sum of all gear ratios is 82818007


In [8]:
import re

file = open('input4.txt', 'r')
cards = []
for line in file.readlines():
    match = re.search('Card +([0-9]+): +(.*)', line)
    winning_numbers, card_numbers = match.group(2).split('|')

    winning_numbers = winning_numbers.split(' ')
    winning_numbers = [int(number) for number in winning_numbers if number.isdigit()]

    card_numbers = card_numbers.split(' ')
    card_numbers = [int(number) for number in card_numbers if number.isdigit()]

    cards.append({
        'winning_numbers': winning_numbers,
        'card_numbers': card_numbers
    })

points = 0
for card in cards:
    count = 0
    for number in card['card_numbers']:
        if number in card['winning_numbers']:
            count += 1
    if count > 0:
        points += 2**(count-1)

    card['count'] = count

print(f'The sum of scratch card points is {points}')

number_of_cards = [1 for card in cards]
for i in range(len(cards)):
    count = cards[i]['count']
    for j in range(count):
        number_of_cards[i+j+1] += number_of_cards[i]

print(f'The total number of scratch cards at the end is {sum(number_of_cards)}')

The sum of scratch card points is 19135
The total number of scratch cards at the end is 5704953


In [29]:
inputs = open('input5.txt', 'r').read().split('\n\n')

# Start with a map from the seed ranges to itself
seeds = inputs[0].split()[1:]
maps = []
for i in range(0, len(seeds), 2):
    start_range = int(seeds[i])
    length = int(seeds[i+1])
    end_range = start_range + length - 1
    maps.append({'source_range': (start_range, end_range), 'transformation': 0})
    # Add single number ranges to solve part A
    maps.append({'source_range': (length, length), 'transformation': 0})

# Go through the mappings one at a time, and append them to the total map
for mapping in inputs[1:]:
    new_maps = []
    for single_map in mapping.strip().split('\n')[1:]:
        target, source, length = [int(x) for x in single_map.split()]
        source_range = (source, source + length - 1)
        transformation = target - source
        new_maps.append({'source_range': source_range, 'transformation': transformation})

    maps = append_maps(maps, new_maps)

def apply_map(start_value, maps):
    for map in maps:
        if map['source_range'][0] <= start_value <= map['source_range'][1]:
            return start_value + map['transformation']
    return start_value

print(f'The lowest reachable location taking the seeds as discrete locations if {min(apply_map(int(seed), maps) for seed in seeds)}')
print(f'The lowest reachable location is {min(map["source_range"][0] + map["transformation"] for map in maps)}')

# Create a set of maps from seed to location, by appending each intermediate map to the running total of maps
def append_maps(original_maps, appended_maps):
    result_maps = []
    has_intersections = True

    while has_intersections:
        has_intersections = False
        for i in range(len(original_maps)):
            original_map = original_maps[i]
            transformation = original_map['transformation']
            target_range = tuple(x + transformation for x in original_map['source_range'])

            for appended_map in appended_maps:
                intersection, non_intersected_sections = find_intersections(target_range, appended_map['source_range'])
                if intersection is not None:
                    has_intersections = True
                    del original_maps[i]

                    appended_transformation = appended_map['transformation']
                    new_source_range = tuple(x - transformation for x in intersection)
                    result_maps.append({'source_range': new_source_range, 'transformation': transformation + appended_transformation})
                    for section in non_intersected_sections:
                        section_range = tuple(x - transformation for x in section)
                        original_maps.append({'source_range': section_range, 'transformation': transformation})
                    break

            if has_intersections:
                break

    # If anything remains in the original maps list at this point, it had no overlaps with the appended_maps
    for original_map in original_maps:
        result_maps.append(original_map)
    return result_maps

def find_intersections(range1, range2):
    intersection = (max(range1[0], range2[0]), min(range1[1], range2[1]))
    if intersection[1] < intersection[0]:
        intersection = None
    non_intersecting = set()
    if range1[0] < range2[0]:
        non_intersecting.add((range1[0], min(range1[1], range2[0]-1)))
    if range2[1] < range1[1]:
        non_intersecting.add((max(range1[0], range2[1]+1), range1[1]))

    return intersection, non_intersecting

The lowest reachable location taking the seeds as discrete locations if 662197086
The lowest reachable location is 52510809


In [5]:
import numpy
import math

def find_wins(time, record):
    return time - 2 * (math.floor((time - math.sqrt(time**2 - 4 * record)) / 2 + 1)) + 1

file = open('input6.txt', 'r')
times_string, distances_string = file.readlines()
times = [int(x) for x in times_string.split() if x.isdigit()]
distances = [int(x) for x in distances_string.split() if x.isdigit()]

wins = [find_wins(time, record) for time, record in zip(times, distances)]
print(f'The product of the numbers of ways to win the short races is {numpy.prod(wins)}')

# One long race
time = int(''.join(x for x in times_string if x.isdigit()))
record = int(''.join(x for x in distances_string if x.isdigit()))
print(f'The number of ways to win the really long race is {find_wins(time, record)}')

The product of the numbers of ways to win the short races is 1624896
The number of ways to win the really long race is 32583852


In [25]:
import functools

hands = [line.split(' ') for line in open('input7.txt', 'r').readlines()]
hands = [(x, int(y)) for x,y in hands]

card_values = ['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2']
with_joker_card_values = ['A', 'K', 'Q', 'T', '9', '8', '7', '6', '5', '4', '3', '2', 'J']

def get_hand_score(hand):
    cards = {}
    for card in hand[0]:
        if card not in cards:
            cards[card] = 0
        cards[card] += 1

    # Check for five of a kind
    if 5 in cards.values():
        return 19
    if 4 in cards.values():
        return 18
    if 3 in cards.values() and 2 in cards.values():
        return 17
    if 3 in cards.values():
        return 16

    pairs = sum(1 if value == 2 else 0 for value in cards.values())
    if pairs == 2:
        return 15
    if pairs == 1:
        return 14

    return 13

def compare_hands(hand1, hand2):
    comparison = get_hand_score(hand1) - get_hand_score(hand2)

    card_index = 0
    while comparison == 0 and card_index < 5:
        comparison = card_values.index(hand2[0][card_index]) - card_values.index(hand1[0][card_index])
        card_index += 1
    return comparison

def get_joker_hand_score(hand):
    cards = {}
    jokers = 0
    for card in hand[0]:
        if card == 'J':
            jokers += 1
            continue
            
        if card not in cards:
            cards[card] = 0
        cards[card] += 1

    if max(cards.values(), default=0) + jokers == 5
        return 19
    if max(cards.values(), default=0) + jokers == 4:
        return 18

    pairs = sum(1 if value == 2 else 0 for value in cards.values())
    if (3 in cards.values() and 2 in cards.values()) or (pairs == 2 and jokers == 1)
        return 17

    if max(cards.values(), default=0) + jokers == 3:
        return 16

    if pairs == 2:
        return 15

    if pairs == 1 or jokers == 1:
        return 14

    return 13

def compare_joker_hands(hand1, hand2):
    comparison = get_joker_hand_score(hand1) - get_joker_hand_score(hand2)

    card_index = 0
    while comparison == 0 and card_index < 5:
        comparison = with_joker_card_values.index(hand2[0][card_index]) - with_joker_card_values.index(hand1[0][card_index])
        card_index += 1
    return comparison

hands.sort(key=functools.cmp_to_key(compare_hands))

winnings = 0
for index, hand in enumerate(hands):
    winnings += (index+1) * hand[1]
print(f'The total winnings for the hands is {winnings}')

hands.sort(key=functools.cmp_to_key(compare_joker_hands))

winnings = 0
for index, hand in enumerate(hands):
    winnings += (index+1) * hand[1]
printf(

The total winnings for the hands is 249638405
